In [1]:
from azureml.core import Workspace
from azureml.core.authentication import MsiAuthentication
msi_auth = MsiAuthentication()

ws = Workspace(subscription_id="7704663e-8944-4300-b951-6462d41ab638", # put in .json
resource_group="arg-rt-pacedev-SpaceTech", # also in .json
workspace_name="aml-WS-spacetech001", # also
auth=msi_auth
)

keyvault = ws.get_default_keyvault()
conn_str = keyvault.get_secret("asartpacedevstvdsm001-connstr")

# Baseline Dataset

In [12]:
import datetime
from dateutil.relativedelta import relativedelta
from vdsm_util import get_blob_client
import pickle as pkl
from azure.storage.blob import BlobServiceClient
import pandas as pd
from io import StringIO
import traceback

veg_type = 'bs'

training_path = 'training/traininginput_{}_v2'.format(veg_type)

blob_client = BlobServiceClient.from_connection_string(conn_str)
container_client = blob_client.get_container_client("data")

baseline_df = pd.DataFrame()

blob_names = container_client.list_blobs(name_starts_with=f'training/traininginput_{veg_type}_v2')

for blob_name in blob_names:
    print(blob_name['name'])
    blob_client = get_blob_client(filename=blob_name, conn_str=conn_str)
    df = pd.read_csv(StringIO(blob_client.download_blob().content_as_text()))
    baseline_df = baseline_df.append(df)

baseline_df.drop(columns=['actual_3m', 'actual_6m', 'actual_9m', 'actual_12m'])


training/traininginput_bs_v2/BM; TP; YA_251_251.csv
training/traininginput_bs_v2/BM_252_256.csv
training/traininginput_bs_v2/BM_257_261.csv
training/traininginput_bs_v2/BM_262_266.csv
training/traininginput_bs_v2/BM_267_271.csv
training/traininginput_bs_v2/BM_272_275.csv
training/traininginput_bs_v2/BM_276_278.csv
training/traininginput_bs_v2/BM_279_283.csv
training/traininginput_bs_v2/BM_284_287.csv
training/traininginput_bs_v2/BM_288_289.csv
training/traininginput_bs_v2/BM_290_291.csv
training/traininginput_bs_v2/BM_291_293.csv
training/traininginput_bs_v2/BM_293_295.csv
training/traininginput_bs_v2/BM_296_300.csv
training/traininginput_bs_v2/BM_301_305.csv
training/traininginput_bs_v2/BM_306_310.csv
training/traininginput_bs_v2/BM_311_315.csv
training/traininginput_bs_v2/BM_316_320.csv
training/traininginput_bs_v2/BM_321_325.csv
training/traininginput_bs_v2/BM_326_328.csv
training/traininginput_bs_v2/DD_-3_0.csv
training/traininginput_bs_v2/DD_103_107.csv
training/traininginput_bs_v

In [52]:
output = baseline_df.to_csv(index=False, encoding='utf-8', header=True)

blob_client = get_blob_client(filename=f'data_drift/baseline/{veg_type}.csv', conn_str=conn_str)
blob_client.upload_blob(output, overwrite=True, blob_type='BlockBlob')

{'etag': '"0x8D9E9EFECCD39D6"',
 'last_modified': datetime.datetime(2022, 2, 7, 4, 11, 32, tzinfo=datetime.timezone.utc),
 'content_md5': None,
 'content_crc64': bytearray(b'\xdb\xf7\x9e\xaf\x02\xe4\xe6\xee'),
 'client_request_id': '089744c0-87cc-11ec-b870-6d68f47935e2',
 'request_id': '4ae79d71-d01e-0043-2cd8-1b2a8e000000',
 'version': '2020-06-12',
 'version_id': '2022-02-07T04:11:32.8974041Z',
 'date': datetime.datetime(2022, 2, 7, 4, 11, 32, tzinfo=datetime.timezone.utc),
 'request_server_encrypted': True,
 'encryption_key_sha256': None,
 'encryption_scope': None}

# Pipeline for monthly target dataset

In [32]:
import datetime
from dateutil.relativedelta import relativedelta
from vdsm_util import get_blob_client
import pickle as pkl
from azure.storage.blob import BlobServiceClient
import pandas as pd
from io import StringIO
import traceback

end_month = '2021-12'
veg_type = 'bs'

blob_client = BlobServiceClient.from_connection_string(conn_str)
container_client = blob_client.get_container_client("data")

end_month_dt = datetime.date(int(end_month[:4]), int(end_month[5:]), 1)
start_month_dt = datetime.date(int(end_month[:4]), int(end_month[5:]), 1) + relativedelta(months=-12)

months = []
month_dt = start_month_dt
while month_dt < end_month_dt:
    month_dt = month_dt + relativedelta(months=1)
    months.append(str(month_dt)[:7])

target_df = pd.DataFrame()

for month in months:

    blob_names = container_client.list_blobs(name_starts_with=f'{month[:4]}/{month[5:]}/fused/{veg_type}_v1')

    for blob_name in blob_names:
        print(blob_name['name'])
        blob_client = get_blob_client(filename=blob_name, conn_str=conn_str)
        df = pd.read_csv(StringIO(blob_client.download_blob().content_as_text()))
        if datetime.date(int(end_month[:4]), int(end_month[5:]), 1) >= datetime.date(2021, 12, 1):
            try:
                df = df.sample(n=100)
            except:
                print(traceback.format_exc())
        target_df = target_df.append(df)

    

2021/01/fused/bs_v1/BM; TP; YA_251_251.csv
2021/01/fused/bs_v1/BM_252_256.csv
2021/01/fused/bs_v1/BM_257_261.csv
2021/01/fused/bs_v1/BM_262_266.csv
2021/01/fused/bs_v1/BM_267_271.csv
2021/01/fused/bs_v1/BM_272_275.csv
2021/01/fused/bs_v1/BM_276_278.csv
2021/01/fused/bs_v1/BM_279_283.csv
2021/01/fused/bs_v1/BM_284_287.csv
2021/01/fused/bs_v1/BM_288_289.csv
2021/01/fused/bs_v1/BM_290_291.csv
2021/01/fused/bs_v1/BM_291_293.csv
2021/01/fused/bs_v1/BM_293_295.csv
2021/01/fused/bs_v1/BM_296_300.csv
2021/01/fused/bs_v1/BM_301_305.csv
2021/01/fused/bs_v1/BM_306_310.csv
2021/01/fused/bs_v1/BM_311_315.csv
2021/01/fused/bs_v1/BM_316_320.csv
2021/01/fused/bs_v1/BM_321_325.csv
2021/01/fused/bs_v1/BM_326_328.csv
2021/01/fused/bs_v1/DD_-3_0.csv
2021/01/fused/bs_v1/DD_103_107.csv
2021/01/fused/bs_v1/DD_108_112.csv
2021/01/fused/bs_v1/DD_113_117.csv
2021/01/fused/bs_v1/DD_118_122.csv
2021/01/fused/bs_v1/DD_123_127.csv
2021/01/fused/bs_v1/DD_128_132.csv
2021/01/fused/bs_v1/DD_133_137.csv
2021/01/fused/b

In [15]:
baseline_df.columns

Index(['year', 'month', 'evi', 'evi_prev', 'u10', 'u10_prev', 'v10',
       'v10_prev', 't2m', 't2m_prev', 'lai_lv', 'lai_lv_prev', 'src',
       'src_prev', 'ssr', 'ssr_prev', 'sp', 'sp_prev', 'e', 'e_prev', 'tp',
       'tp_prev', 'tp_prev2', 'tp_prev3', 'swvl1', 'swvl1_prev', 'co2',
       'co2_prev', 'lat_4326', 'lon_4326', 'season_ch_3mon', 'season_ch_6mon',
       'season_ch_9mon', 'season_ch_12mon', 'actual_3m', 'actual_6m',
       'actual_9m', 'actual_12m'],
      dtype='object')

In [14]:
target_df.columns

Index(['filename', 'lat', 'lon', 'year', 'month1', 'month2', 'month3',
       'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10',
       'month11', 'month12', 'evi', 'evi_prev', 'u10', 'u10_prev', 'v10',
       'v10_prev', 't2m', 't2m_prev', 'lai_lv', 'lai_lv_prev', 'src',
       'src_prev', 'ssr', 'ssr_prev', 'sp', 'sp_prev', 'e', 'e_prev', 'tp',
       'tp_prev', 'tp_prev2', 'tp_prev3', 'swvl1', 'swvl1_prev', 'co2',
       'co2_prev', 'season_ch_3mon', 'season_ch_6mon', 'season_ch_9mon',
       'season_ch_12mon'],
      dtype='object')

In [20]:
target_df.drop(columns=['filename', 'lat', 'lon', 'year'], inplace=True)
target_df.insert(0, 'month', target_df.iloc[:, 0:12].values.argmax(axis=1) + 1)
target_df.drop(columns=['month1','month2','month3','month4','month5','month6','month7','month8','month9','month10','month11','month12'], inplace=True)

month1  month2  month3  month4  month5  month6  month7  month8  month9  \
7        1.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
17       1.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
76       1.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
19       1.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
16       1.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
...      ...     ...     ...     ...     ...     ...     ...     ...     ...   
192      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3712     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3985     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
829      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
521      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

      month10  month11  month12  
7         0.0      0.0      0.0  
17        0.0      0.0      0.0  
76        0.0      0.0      0.0  
19        0.0      0.0      0.0  
16        0.0      0.0      0.0  
...       ...      ...      ...  
192       0.0      0.0      1.0  
3712      0.0      0.0      1.0  
3985      0.0      0.0      1.0  
829       0.0      0.0      1.0  
521       0.0      0.0      1.0  

[298728 rows x 12 columns]

In [23]:
target_df['month'] = target_df.iloc[:, 0:12].values.argmax(axis=1) + 1

In [30]:
# cols = target_df.columns.tolist()
# cols = cols[-1:] + cols[:-1]
# target_df = target_df[cols]
target_df.insert(0, 'month', target_df.iloc[:, 0:12].values.argmax(axis=1) + 1)

In [29]:
target_df.drop(columns=['month'], inplace=True)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [27]:
target_df.drop(columns=['month1','month2','month3','month4','month5','month6','month7','month8','month9','month10','month11','month12'], inplace=True)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [31]:
target_df

month       evi  evi_prev       u10  u10_prev       v10  v10_prev  \
7         7  0.088725  0.113041 -0.372381 -0.739500  0.571238 -0.424562   
17        7  0.154429  0.160743 -0.373166 -0.740331  0.567399 -0.430086   
76        7  0.101994  0.112342 -0.377292 -0.743918  0.567188 -0.428316   
19        7  0.096303  0.101926 -0.372919 -0.740003  0.570632 -0.425206   
16        7  0.092087  0.081871 -0.372733 -0.739861  0.569879 -0.426476   
...     ...       ...       ...       ...       ...       ...       ...   
192       7  0.116215  0.135704  0.148814 -0.314824  0.882098 -0.031735   
3712      7  0.092213  0.088215  0.164636 -0.313497  0.902785 -0.032228   
3985      7  0.135886  0.133433  0.167978 -0.313001  0.906422 -0.032398   
829       7  0.110744  0.110383  0.147781 -0.316536  0.885938 -0.031234   
521       7  0.111715  0.106722  0.154605 -0.313247  0.886255 -0.032224   

             t2m    t2m_prev    lai_lv  ...  tp_prev2      tp_prev3     swvl1  \
7     304.679412  302.486974  0.957414  ...  0.000268  8.823470e-05  0.147910   
17    304.664211  302.478581  0.914730  ...  0.000263  8.619576e-05  0.137376   
76    304.672072  302.481949  0.909132  ...  0.000264  8.613806e-05  0.136004   
19    304.678016  302.486139  0.950417  ...  0.000267  8.792805e-05  0.146190   
16    304.674220  302.484100  0.942278  ...  0.000266  8.751581e-05  0.144174   
...          ...         ...       ...  ...       ...           ...       ...   
192   307.901082  301.036303  1.119227  ...  0.000001  9.555370e-07  0.070581   
3712  307.828994  300.966162  1.099243  ...  0.000001  9.555370e-07  0.068905   
3985  307.813659  300.951238  1.089569  ...  0.000001  9.555370e-07  0.068084   
829   307.906878  301.041100  1.142490  ...  0.000001  9.555370e-07  0.072571   
521   307.873915  301.010615  1.101193  ...  0.000001  9.555370e-07  0.069049   

      swvl1_prev     co2  co2_prev  season_ch_3mon  season_ch_6mon  \
7       0.204256  414.77    414.14        0.030700       -0.043848   
17      0.195085  414.77    414.14        0.066627       -0.023675   
76      0.194095  414.77    414.14        0.097359       -0.049191   
19      0.202781  414.77    414.14        0.017990       -0.052973   
16      0.201007  414.77    414.14       -0.002011       -0.059926   
...          ...     ...       ...             ...             ...   
192     0.090977  414.01    414.01        0.024827       -0.043350   
3712    0.089419  414.01    414.01        0.022487       -0.021303   
3985    0.088639  414.01    414.01        0.064364        0.003678   
829     0.092909  414.01    414.01        0.077511       -0.002942   
521     0.089509  414.01    414.01        0.039187       -0.033448   

      season_ch_9mon  season_ch_12mon  
7          -0.009872        -0.028344  
17          0.010973        -0.000594  
76         -0.022523        -0.025582  
19         -0.021069        -0.018636  
16         -0.012658        -0.004826  
...              ...              ...  
192        -0.029362        -0.040628  
3712       -0.004928         0.003751  
3985       -0.034043        -0.022911  
829         0.000790        -0.019999  
521        -0.010211         0.000228  

[298728 rows x 31 columns]

In [8]:
import datetime
from dateutil.relativedelta import relativedelta
from vdsm_util import get_blob_client
import pickle as pkl
from azure.storage.blob import BlobServiceClient
import pandas as pd
from io import StringIO
import traceback

end_month = '2021-12'
veg_type = 'bs'

training_path = 'training/traininginput_{}_v2'.format(veg_type)

blob_client = BlobServiceClient.from_connection_string(conn_str)
container_client = blob_client.get_container_client("data")

end_month_dt = datetime.date(int(end_month[:4]), int(end_month[5:]), 1)
start_month_dt = datetime.date(int(end_month[:4]), int(end_month[5:]), 1) + relativedelta(months=-12)

months = []
month_dt = start_month_dt
while month_dt < end_month_dt:
    month_dt = month_dt + relativedelta(months=1)
    months.append(str(month_dt)[:7])

target_df = pd.DataFrame()

for month in months:

    blob_names = container_client.list_blobs(name_starts_with=f'{month[:4]}/{month[5:]}/fused/{veg_type}_v1')

    for blob_name in blob_names:
        print(blob_name['name'])
        blob_client = get_blob_client(filename=blob_name, conn_str=conn_str)
        df = pd.read_csv(StringIO(blob_client.download_blob().content_as_text()))
        if datetime.date(int(end_month[:4]), int(end_month[5:]), 1) >= datetime.date(2021, 12, 1):
            try:
                df = df.sample(n=100)
            except:
                print(traceback.format_exc())
        target_df = target_df.append(df)

    

2021/01/fused/bs_v1/BM; TP; YA_251_251.csv
2021/01/fused/bs_v1/BM_252_256.csv
2021/01/fused/bs_v1/BM_257_261.csv
2021/01/fused/bs_v1/BM_262_266.csv
2021/01/fused/bs_v1/BM_267_271.csv
2021/01/fused/bs_v1/BM_272_275.csv
2021/01/fused/bs_v1/BM_276_278.csv
2021/01/fused/bs_v1/BM_279_283.csv
2021/01/fused/bs_v1/BM_284_287.csv
2021/01/fused/bs_v1/BM_288_289.csv
2021/01/fused/bs_v1/BM_290_291.csv
2021/01/fused/bs_v1/BM_291_293.csv
2021/01/fused/bs_v1/BM_293_295.csv
2021/01/fused/bs_v1/BM_296_300.csv
2021/01/fused/bs_v1/BM_301_305.csv
2021/01/fused/bs_v1/BM_306_310.csv
2021/01/fused/bs_v1/BM_311_315.csv
2021/01/fused/bs_v1/BM_316_320.csv
2021/01/fused/bs_v1/BM_321_325.csv
2021/01/fused/bs_v1/BM_326_328.csv
2021/01/fused/bs_v1/DD_-3_0.csv
2021/01/fused/bs_v1/DD_103_107.csv
2021/01/fused/bs_v1/DD_108_112.csv
2021/01/fused/bs_v1/DD_113_117.csv
2021/01/fused/bs_v1/DD_118_122.csv
2021/01/fused/bs_v1/DD_123_127.csv
2021/01/fused/bs_v1/DD_128_132.csv
2021/01/fused/bs_v1/DD_133_137.csv
2021/01/fused/b

In [9]:
target_df

filename        lat         lon    year  month1  month2  \
77    BM; TP; YA_251_251.csv -22.424044  117.694633  2021.0     1.0     0.0   
68    BM; TP; YA_251_251.csv -22.427872  117.696825  2021.0     1.0     0.0   
15    BM; TP; YA_251_251.csv -22.426250  117.695940  2021.0     1.0     0.0   
76    BM; TP; YA_251_251.csv -22.431692  117.700693  2021.0     1.0     0.0   
54    BM; TP; YA_251_251.csv -22.429128  117.696769  2021.0     1.0     0.0   
...                      ...        ...         ...     ...     ...     ...   
3293          YA_447_449.csv -22.786871  119.185599  2021.0     0.0     0.0   
2074          YA_447_449.csv -22.785943  119.197063  2021.0     0.0     0.0   
2931          YA_447_449.csv -22.786491  119.190285  2021.0     0.0     0.0   
2121          YA_447_449.csv -22.785871  119.195293  2021.0     0.0     0.0   
365           YA_447_449.csv -22.778105  119.191014  2021.0     0.0     0.0   

      month3  month4  month5  month6  ...  tp_prev2      tp_prev3     swvl1  \
77       0.0     0.0     0.0     0.0  ...  0.000269  8.845857e-05  0.149074   
68       0.0     0.0     0.0     0.0  ...  0.000266  8.735526e-05  0.142909   
15       0.0     0.0     0.0     0.0  ...  0.000267  8.782168e-05  0.145517   
76       0.0     0.0     0.0     0.0  ...  0.000264  8.613806e-05  0.136004   
54       0.0     0.0     0.0     0.0  ...  0.000266  8.701910e-05  0.141192   
...      ...     ...     ...     ...  ...       ...           ...       ...   
3293     0.0     0.0     0.0     0.0  ...  0.000001  9.555370e-07  0.069815   
2074     0.0     0.0     0.0     0.0  ...  0.000001  9.555370e-07  0.073536   
2931     0.0     0.0     0.0     0.0  ...  0.000001  9.555370e-07  0.071351   
2121     0.0     0.0     0.0     0.0  ...  0.000001  9.555370e-07  0.072896   
365      0.0     0.0     0.0     0.0  ...  0.000001  9.555370e-07  0.068953   

      swvl1_prev     co2  co2_prev  season_ch_3mon  season_ch_6mon  \
77      0.205268  414.77    414.14        0.038925       -0.005461   
68      0.199976  414.77    414.14        0.049474       -0.045673   
15      0.202215  414.77    414.14        0.020787       -0.039066   
76      0.194095  414.77    414.14        0.097359       -0.049191   
54      0.198478  414.77    414.14        0.034046       -0.050047   
...          ...     ...       ...             ...             ...   
3293    0.090285  414.01    414.01        0.047641       -0.006385   
2074    0.093859  414.01    414.01        0.050558       -0.006488   
2931    0.091761  414.01    414.01        0.051817       -0.002726   
2121    0.093243  414.01    414.01        0.147988        0.009344   
365     0.089412  414.01    414.01        0.014366       -0.044689   

      season_ch_9mon  season_ch_12mon  
77          0.004184        -0.002939  
68         -0.015483         0.022025  
15         -0.012228        -0.005365  
76         -0.022523        -0.025582  
54         -0.026969        -0.006280  
...              ...              ...  
3293       -0.036075        -0.018385  
2074        0.008271        -0.004148  
2931       -0.004817         0.003601  
2121       -0.048497        -0.050057  
365        -0.025151        -0.007804  

[298728 rows x 46 columns]

In [10]:
output = target_df.to_csv(index=False, encoding='utf-8', header=True)

blob_client = get_blob_client(filename=f'data_drift/target/2021-12/{veg_type}.csv', conn_str=conn_str)
blob_client.upload_blob(output, overwrite=True, blob_type='BlockBlob')

{'etag': '"0x8D9EAB064F03616"',
 'last_modified': datetime.datetime(2022, 2, 8, 3, 9, 17, tzinfo=datetime.timezone.utc),
 'content_md5': None,
 'content_crc64': bytearray(b'\x08\x8e\xed\xb6k\x86T\xda'),
 'client_request_id': '80b9c80e-888c-11ec-8670-95a8cb31c425',
 'request_id': '673bc1c9-601e-003c-1a99-1cb4bc000000',
 'version': '2020-06-12',
 'version_id': '2022-02-08T03:09:17.8243606Z',
 'date': datetime.datetime(2022, 2, 8, 3, 9, 17, tzinfo=datetime.timezone.utc),
 'request_server_encrypted': True,
 'encryption_key_sha256': None,
 'encryption_scope': None}